In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
pip install mne

Note: you may need to restart the kernel to use updated packages.


In [2]:
!wget -r -N -c -np https://physionet.org/files/eegmat/1.0.0/

--2024-07-05 16:56:58--  https://physionet.org/files/eegmat/1.0.0/
Resolving physionet.org (physionet.org)... 18.18.42.54
Connecting to physionet.org (physionet.org)|18.18.42.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: 'physionet.org/files/eegmat/1.0.0/index.html'

physionet.org/files     [ <=>                ]   9.10K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2024-07-05 16:56:58 (187 MB/s) - 'physionet.org/files/eegmat/1.0.0/index.html' saved [9320]

Loading robots.txt; please ignore errors.
--2024-07-05 16:56:58--  https://physionet.org/robots.txt
Reusing existing connection to physionet.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 22 [text/plain]
Saving to: 'physionet.org/robots.txt'

physionet.org/robot 100%[===================>]      22  --.-KB/s    in 0s      

2024-07-05 16:56:58 (3.14 MB/s) - 'physionet.org/robots.txt' saved [22/22]

--2024-07-05 16:56:

In [3]:
import os
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
num_subjects = 36
file_path_template = '/kaggle/working/physionet.org/files/eegmat/1.0.0/Subject{subject_number:02d}_{recording_type}.edf'

# Function to generate file paths
def get_file_paths(num_subjects, file_path_template):
    file_paths = []
    for subject_number in range(num_subjects):
        for recording_type in [1, 2]:  # 1 for rest, 2 for task
            file_path = file_path_template.format(subject_number=subject_number, recording_type=recording_type)
            file_paths.append((subject_number, recording_type, file_path))
    return file_paths

file_paths = get_file_paths(num_subjects, file_path_template)

In [5]:
file_paths

[(0, 1, '/kaggle/working/physionet.org/files/eegmat/1.0.0/Subject00_1.edf'),
 (0, 2, '/kaggle/working/physionet.org/files/eegmat/1.0.0/Subject00_2.edf'),
 (1, 1, '/kaggle/working/physionet.org/files/eegmat/1.0.0/Subject01_1.edf'),
 (1, 2, '/kaggle/working/physionet.org/files/eegmat/1.0.0/Subject01_2.edf'),
 (2, 1, '/kaggle/working/physionet.org/files/eegmat/1.0.0/Subject02_1.edf'),
 (2, 2, '/kaggle/working/physionet.org/files/eegmat/1.0.0/Subject02_2.edf'),
 (3, 1, '/kaggle/working/physionet.org/files/eegmat/1.0.0/Subject03_1.edf'),
 (3, 2, '/kaggle/working/physionet.org/files/eegmat/1.0.0/Subject03_2.edf'),
 (4, 1, '/kaggle/working/physionet.org/files/eegmat/1.0.0/Subject04_1.edf'),
 (4, 2, '/kaggle/working/physionet.org/files/eegmat/1.0.0/Subject04_2.edf'),
 (5, 1, '/kaggle/working/physionet.org/files/eegmat/1.0.0/Subject05_1.edf'),
 (5, 2, '/kaggle/working/physionet.org/files/eegmat/1.0.0/Subject05_2.edf'),
 (6, 1, '/kaggle/working/physionet.org/files/eegmat/1.0.0/Subject06_1.edf'),

In [6]:
# Function to load and preprocess a single file
def load_and_preprocess(file_path):
    raw = mne.io.read_raw_edf(file_path, preload=True)
    raw.set_eeg_reference()
    epochs = mne.make_fixed_length_epochs(raw, duration=60)
    raw = epochs.average()
    raw = raw.get_data()
    return raw

# Dictionary to store raw data for each subject and recording type
raw_data = {}

for subject_number, recording_type, file_path in file_paths:
    raw = load_and_preprocess(file_path)
    raw_data[(subject_number, recording_type)] = raw


Extracting EDF parameters from /kaggle/working/physionet.org/files/eegmat/1.0.0/Subject00_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Not setting metadata
3 matching events found
No baseline correction applied
0 projection items activated
Extracting EDF parameters from /kaggle/working/physionet.org/files/eegmat/1.0.0/Subject00_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Extracting EDF parameters from /kaggle/working/physionet.org/files/eegmat/1.0.0/Subject01_1.

In [7]:
def get_data_matrix(raw_data):
  values_list = list(raw_data.values())

  # Step 2: Stack the values into a single NumPy array
  stacked_array = np.stack(values_list)

  # Step 3: Reshape the array to the desired dimensions (32, 1, 21, 30000)
  reshaped_array = stacked_array[:, np.newaxis, :, :]

  # Print the shape to verify
  return(reshaped_array)

In [8]:
def get_labels(raw_data):
  result_array = np.zeros(len(raw_data))

# Fill the NumPy array based on the dictionary keys
  for idx, key in enumerate(raw_data.keys()):
      if key[1] == 2:
          result_array[idx] = 1


  return(result_array)

In [9]:
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

In [16]:
class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = 21
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 30000), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 120 timepoints. 
        self.fc1 = nn.Linear(4*2, 1)

    def forward(self, x):
        # Layer 1
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        x = x.permute(0, 3, 1, 2)
        
        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.25)
        x = self.pooling2(x)
        
        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, 0.25)
        x = self.pooling3(x)
        
        # FC Layer
        # print(x.shape)
        x = x.reshape(-1, 4*2)
#         print("a" , x)
#         print("b" ,self.fc1(x))
        x = F.sigmoid(self.fc1(x))
        return x



net = EEGNet().cuda(0)
print(net.forward(Variable(torch.Tensor(np.random.rand(1, 1, 21, 30000)).cuda(0))))
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters())
    

      
    

a tensor([[0.8351, 3.0467, 0.9617, 0.8714, 3.7253, 2.2131, 3.0596, 0.5280]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>)
b tensor([[-0.0347]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.4913]], device='cuda:0', grad_fn=<SigmoidBackward0>)


In [15]:
def evaluate(model, X, Y, params = ["acc"]):
    results = []
    batch_size = 100
    
    predicted = []
    
    for i in range(len(X)//batch_size):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(torch.from_numpy(X[s:e]).cuda(0)) # Uncomment for GPU and comment below line.
        inputs = Variable(torch.from_numpy(X[s:e]))
        pred = model(inputs)
        
        predicted.append(pred.data.cpu().numpy())
        
        
#     inputs = Variable(torch.from_numpy(X).cuda(0)) # Uncomment for GPU and comment below line.
    inputs = Variable(torch.from_numpy(X))
    predicted = model(inputs)
    
    predicted = predicted.data.cpu().numpy()
    
    for param in params:
        if param == 'acc':
            results.append(accuracy_score(Y, np.round(predicted)))
        if param == "auc":
            results.append(roc_auc_score(Y, predicted))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted)))
        if param == "precision":
            results.append(precision_score(Y, np.round(predicted)))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted))
            recall = recall_score(Y, np.round(predicted))
            results.append(2*precision*recall/ (precision+recall))
    return results

In [11]:
X = get_data_matrix(raw_data).astype('float32')
y = get_labels(raw_data).astype('float32')
X.shape , y.shape

((72, 1, 21, 30000), (72,))

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape , y_train.shape , X_val.shape , y_val.shape

((57, 1, 21, 30000), (57,), (15, 1, 21, 30000), (15,))

In [ ]:
batch_size = 8
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters())
    
for epoch in range(100):  # loop over the dataset multiple times
    print("\nEpoch ", epoch)
    
    running_loss = 0.0
    for i in range(len(X_train)//batch_size-1):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = torch.from_numpy(X_train[s:e])
        labels = torch.FloatTensor(np.array([y_train[s:e]]).T*1.0)
        
        # wrap them in Variable
        inputs, labels = Variable(inputs.cuda(0)), Variable(labels.cuda(0))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
#         print("loss" , loss)
#         print(loss)
        running_loss += loss.item() 
        loss.backward()
        
        
        optimizer.step()
        
        # running_loss += loss.data[0]
    
    # Validation accuracy
    params = ["acc", "auc", "fmeasure"]
    print(params)
    print("Training Loss ", running_loss)
    print("Train - ", evaluate(net, X_train, y_train, params))
    print("Validation - ", evaluate(net, X_val, y_val, params))

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score

class ATCNet(nn.Module):
    def __init__(self, num_classes=1, num_channels=21, input_length=30000):
        super(ATCNet, self).__init__()
        
        # Temporal convolutional layers
        self.temporal_conv1 = nn.Conv1d(num_channels, 64, kernel_size=5, padding=2)
        self.batchnorm1 = nn.BatchNorm1d(64, False)
        self.temporal_conv2 = nn.Conv1d(64, 128, kernel_size=5, padding=2)
        self.batchnorm2 = nn.BatchNorm1d(128, False)

        # Attention layer
        self.attention = nn.MultiheadAttention(embed_dim=128, num_heads=4)

        # Fully connected layer for classification
        self.fc = nn.Linear(128 * input_length,1)

    def forward(self, x):
        x = x.squeeze(1)  # Remove the single channel dimension

        # Apply temporal convolutions
        x = F.relu(self.temporal_conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        x = F.relu(self.temporal_conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.25)

        # Apply attention mechanism
        x = x.permute(2, 0, 1)  # Rearrange dimensions for attention (seq_len, batch, feature)
        x, _ = self.attention(x, x, x)
        x = x.permute(1, 2, 0)  # Rearrange dimensions back (batch, feature, seq_len)

        # Flatten and pass through fully connected layer
        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)
        x = F.sigmoid(x)

        return x


In [ ]:
# Hyperparameters
batch_size = 1
learning_rate = 0.001
num_epochs = 10

# Model, criterion, optimizer
# net = ATCNet().cuda(0) # # Uncomment for GPU and comment below line.
net = ATCNet()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

# Training data (assuming X_train, y_train, X_val, y_val are defined)
for epoch in range(num_epochs):
    print("\nEpoch ", epoch)
    running_loss = 0.0
    
    for i in range(len(X_train) // 16):
        s = i * batch_size
        e = i * batch_size + batch_size
        
        inputs = torch.from_numpy(X_train[s:e]).float()
        labels = torch.FloatTensor(np.array([y_train[s:e]]).T * 1.0)
#         print(labels)
        
        # Wrap them in Variable
#         inputs, labels = Variable(inputs.cuda(0)), Variable(labels.cuda(0))# Uncomment for GPU and comment below line.
    
        inputs, labels = Variable(inputs), Variable(labels)
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        print("loss:", loss.item())
        running_loss += loss.item()
        loss.backward()
        
        optimizer.step()
        
    # Validation accuracy
    params = ["acc", "auc", "fmeasure"]
    print(params)
    print("Training Loss:", running_loss)
    print("Train -", evaluate(net, X_train, y_train, params))
    print("Validation -", evaluate(net, X_val, y_val, params))


<!-- CPU -->